# Imports

In [ ]:
import os
import pandas as pd

from recoxplainer.config import cfg
from recoxplainer.data_reader.data_reader import DataReader
from recoxplainer.models import ALS
from recoxplainer.evaluator import Splitter, Evaluator
from recoxplainer.recommender.recommender import Recommender


# 2. Load Data


In [5]:
data = DataReader(**cfg.data.ml100k)

# Re-arrange users' and items' Ids
data.make_consecutive_ids_in_dataset()

# Because ALS works on implicit feedback we need to binarize it:+
data.binarize(binary_threshold=1)

# Prepare train and test sets:
sp = Splitter()
train, test = sp.split_leave_n_out(data, frac=0.1)

movie_ids = data.dataset["itemId"].unique()
all_groups = data.read_groups('groupsWithHighRatings5')

c:\Users\usuar\miniconda3\envs\recoxplainer\lib\site-packages\pandas\core\generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
c:\Users\usuar\miniconda3\envs\recoxplainer\lib\site-packages\pandas\core\generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


# Process each group

In [ ]:
for group_id in all_groups:
    group_id = group_id.strip('\n')
    members = data.parse_group_members(group_id)
    print(f"Processing group: {members}")
    
    ## Get candidate items for recommendatioGn
    candidate_movies = data.get_items_for_group_recommendation(data.dataset, movie_ids, members)

    ## Train model
    model = ALS(**cfg.model.als)
    model.fit(train)

    # 4. Generate Recommendations
    recommender = Recommender(model)
    target_item = recommender.generate_group_recommendations(members, candidate_movies, 0)

NameError: name 'all_groups' is not defined